In [48]:
import os
from typing import Tuple, Callable, Dict, Optional, List

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Recommenders.DataIO import DataIO
from skopt.space import Real, Integer, Categorical
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender,MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
np.random.seed(2498)


columns_name = ['user_id','item_id','impression_list','data']

dataset_l = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_length.csv', sep=',')
dataset_t = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_ICM_type.csv', sep=',')
dataset = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/interactions_and_impressions.csv', sep=',')
test_users = pd.read_csv('/Users/loren/Documents/GitHub/RecSys/dati/data_target_users_test.csv',sep = ',')
dataset.columns = columns_name

c:\Users\loren\anaconda3\envs\RecSysFramework\lib\site-packages\IPython\core\interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [49]:
len(dataset.item_id.unique()),dataset.item_id.unique().min(),dataset.item_id.unique().max()

(24507, 0, 24506)

In [50]:
from Data_manager.Build_Matrix.Data_import import build_ICM_type,build_ICM_lengh,build_URM_ICM_onlyURM_item
import scipy.sparse as sps
URM_ALL,ICM=build_URM_ICM_onlyURM_item(dataset, dataset_t, dataset_l)
ICM_TYPE   =build_ICM_type(dataset_t)
ICM_LENGHT =build_ICM_lengh(dataset_l)
ICM_genre_subgenre = sps.hstack((ICM_TYPE, ICM_LENGHT));

In [51]:
rows = int(41629/3)
rows

13876

In [52]:
URM_slice = URM_ALL[rows:rows*2]

In [53]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_slice, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 202 ( 1.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 81 ( 0.6%) Users that have less than 1 test interactions


In [54]:
param_1 = {'l1_ratio': 5.371794099882435e-05, 'topK': 1651, 'alpha': 0.008001600311161093, 'workers': 2, 'mw': 1}

In [55]:
import os

output_folder_path = "result_experiments_SLIMElasticNetRecommender_URM_slice_second_part"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 500
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

hyperparameters_range_dictionary = {
        "l1_ratio": Real(low = 1e-5, high = 0.1, prior = 'log-uniform'),
        "topK": Integer(200, 6000),
        "alpha": Real(low = 1e-5, high = 0.1, prior = 'uniform'),
        "workers": Categorical([2]),
        "mw": Integer(0, 10),
    }

earlystopping_keywargs = {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": URM_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": metric_to_optimize,
                              }
recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [URM_train,ICM_TYPE],
                CONSTRUCTOR_KEYWORD_ARGS = {},
                FIT_POSITIONAL_ARGS = [],
                FIT_KEYWORD_ARGS = {},
                EARLYSTOPPING_KEYWORD_ARGS = {},
            )

recommender_class = MultiThreadSLIM_SLIM_S_ElasticNetRecommender


hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                        evaluator_validation=evaluator_validation,
                                        evaluator_test=evaluator_test)

In [56]:
hyperparameterSearch.search(recommender_input_args,
                    hyperparameter_search_space = hyperparameters_range_dictionary,
                    n_cases = n_cases,
                    n_random_starts = n_random_starts,
                    save_model = "last",
                    output_folder_path = output_folder_path, # Where to save the results
                    output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                    metric_to_optimize = metric_to_optimize,
                    cutoff_to_optimize = cutoff_to_optimize,
                    resume_from_saved = True,
                    recommender_input_args_last_test=recommender_input_args
                    )

SearchBayesianSkopt: Resuming 'SLIMElasticNetRecommender' Failed, no such file exists.



  0%|          | 0/24507 [00:00<?, ?it/s]

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.0012348546776512251, 'topK': 1459, 'alpha': 0.06964542194937692, 'workers': 2, 'mw': 5}
SLIMElasticNetRecommender: URM Detected 18 ( 0.1%) items with no interactions.


100%|█████████▉| 24504/24507 [22:53<00:00, 19.59it/s]

EvaluatorHoldout: Processed 13674 (100.0%) in 22.17 sec. Users per second: 617
SearchBayesianSkopt: New best config found. Config 0: {'l1_ratio': 0.0012348546776512251, 'topK': 1459, 'alpha': 0.06964542194937692, 'workers': 2, 'mw': 5} - results: PRECISION: 0.0436229, PRECISION_RECALL_MIN_DEN: 0.0688891, RECALL: 0.0635791, MAP: 0.0191777, MAP_MIN_DEN: 0.0300562, MRR: 0.1373283, NDCG: 0.0648991, F1: 0.0517435, HIT_RATE: 0.3117595, ARHR_ALL_HITS: 0.1616262, NOVELTY: 0.0039542, AVERAGE_POPULARITY: 0.2992674, DIVERSITY_MEAN_INTER_LIST: 0.8565554, DIVERSITY_HERFINDAHL: 0.9856493, COVERAGE_ITEM: 0.1111519, COVERAGE_ITEM_HIT: 0.0159546, ITEMS_IN_GT: 0.8586118, COVERAGE_USER: 0.9854425, COVERAGE_USER_HIT: 0.3072211, USERS_IN_GT: 0.9854425, DIVERSITY_GINI: 0.0081444, SHANNON_ENTROPY: 7.4394439, RATIO_DIVERSITY_HERFINDAHL: 0.9859714, RATIO_DIVERSITY_GINI: 0.0168545, RATIO_SHANNON_ENTROPY: 0.5523059, RATIO_AVERAGE_POPULARITY: 5.3380808, RATIO_NOVELTY: 0.2689395, 



...

Iteration No: 58 ended. Evaluation done at random point.
Time taken: 4371.1893
Function value obtained: -0.0187
Current minimum: -0.0218
Iteration No: 59 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 0.00010390841624027083, 'topK': 2864, 'alpha': 0.012060199573983242, 'workers': 2, 'mw': 5}
SLIMElasticNetRecommender: URM Detected 18 ( 0.1%) items with no interactions.


EvaluatorHoldout: Processed 13674 (100.0%) in 22.86 sec. Users per second: 598
SearchBayesianSkopt: Config 58 is suboptimal. Config: {'l1_ratio': 0.00010390841624027083, 'topK': 2864, 'alpha': 0.012060199573983242, 'workers': 2, 'mw': 5} - results: PRECISION: 0.0477402, PRECISION_RECALL_MIN_DEN: 0.0746414, RECALL: 0.0687568, MAP: 0.0213123, MAP_MIN_DEN: 0.0332400, MRR: 0.1497012, NDCG: 0.0709791, F1: 0.0563528, HIT_RATE: 0.3342840, ARHR_ALL_HITS: 0.1777923, NOVELTY: 0.0041003, AVERAGE_POPULARITY: 0.2533488, DIVERSITY_MEAN_INTER_LIST: 0.9075243, DIVERSITY_HERFINDAHL: 0.9907458, COVERAGE_ITEM: 0.1324111, COVERAGE_ITEM_HIT: 0.0225650, ITEMS_IN_GT: 0.8586118, COVERAGE_USER: 0.9854425, COVERAGE_USER_HIT: 0.3294177, USERS_IN_GT: 0.9854425, DIVERSITY_GINI: 0.0124567, SHANNON_ENTROPY: 8.1571508, RATIO_DIVERSITY_HERFINDAHL: 0.9910696, RATIO_DIVERSITY_GINI: 0.0257785, RATIO_SHANNON_ENTROPY: 0.6055886, RATIO_AVERAGE_POPULARITY: 4.5190241, RATIO_NOVELTY: 0.2788736, 



Iteration No: 59 ended. Evaluation done at random point.
Time taken: 1321.8547
Function value obtained: -0.0213
Current minimum: -0.0218
Iteration No: 60 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'l1_ratio': 2.9316019928027146e-05, 'topK': 2606, 'alpha': 0.009883295106176073, 'workers': 2, 'mw': 3}
SLIMElasticNetRecommender: URM Detected 18 ( 0.1%) items with no interactions.




















































































































































































































100%|█████████▉| 24504/24507 [15:49:16<00:06,  2.32s/it]


























































































































































































































































































































































































































































































































































































KeyboardInterrupt: 

In [ ]:
from Recommenders.DataIO import DataIO
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
import pandas as pd
recommender_class = SLIMElasticNetRecommender

output_folder_path = "result_experiments"
data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")
time_df = search_metadata["time_df"]

print('mean-time:')
print(time_df['train'].mean())


result_on_test_df = search_metadata["result_on_test_df"]
result_best_on_test = search_metadata["result_on_last"]
print('BEST_HYPERPARAMETERS: ')
print(search_metadata["hyperparameters_best"])
print(pd.DataFrame(result_on_test_df))
print(result_best_on_test)

mean-time:
392.422853257921
BEST_HYPERPARAMETERS: 
{'topK': 668, 'l1_ratio': 0.06262713223835095, 'alpha': 0.001}
          PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
   cutoff                                                                      
0  10      0.048502                 0.067374  0.059069  0.022471    0.030846   
1  10           NaN                      NaN       NaN       NaN         NaN   
2  10           NaN                      NaN       NaN       NaN         NaN   
3  10           NaN                      NaN       NaN       NaN         NaN   
4  10           NaN                      NaN       NaN       NaN         NaN   
...             ...                      ...       ...       ...         ...   
95 10           NaN                      NaN       NaN       NaN         NaN   
96 10           NaN                      NaN       NaN       NaN         NaN   
97 10           NaN                      NaN       NaN       NaN         NaN   
98 10 